In [4]:
import requests
import json
import time
import pandas as pd
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [8]:
# API calls
headers = {
    'Accept': 'application/json',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,id;q=0.7,ms;q=0.6',
    'Cache-Control': 'no-cache',
    'Origin': 'https://www2.hm.com',
    'Pragma': 'no-cache',
    'Priority': 'u=1, i',
    'Referer': 'https://www2.hm.com/',
    'Sec-Ch-Ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    'X-Customer-Key': 'd150a6cc-6e04-4804-8447-3ab67460a1d9',
    'X-Session-Key': 'd150a6cc-6e04-4804-8447-3ab67460a1d9',
}

# Base URL
product_list_base_url = 'https://api.hm.com/search-services/v1/en_us/listing/resultpage'

# Base URL supplier details GET request
supplier_detail_base_url = 'https://www2.hm.com/en_us/supplierDetails/articles/'

# Base parameters
base_params = {
    'pageSource': 'PLP',
    'page': 1, 
    'sort': 'RELEVANCE',
    'pageId': '/men/shop-by-product/view-all',
    'page-size': 36,
    'categoryId': 'men_viewall',
    'filters': 'sale:false||oldSale:false',
    'touchPoint': 'DESKTOP',
    'skipStockCheck': 'false',
}

# Max retries for a request
RETRY_MAX = 5
# Status codes to retry on (400 Client Error, 5xx Server Errors, etc)
RETRY_STATUS_FORCELIST = [400, 401, 403, 404, 408, 429, 500, 502, 503, 504]
RETRY_BACKOFF_FACTOR = 0.5 # 0.5, 1, 2, 4, 8 seconds

session = requests.Session()
retries = Retry(total=RETRY_MAX,
                backoff_factor=RETRY_BACKOFF_FACTOR,
                status_forcelist=RETRY_STATUS_FORCELIST,
                allowed_methods=frozenset(['GET', 'POST']))

session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))


# --- Product Listing Scraping Loop ---
all_products_raw = []
current_page = 1
total_pages = 1 

while current_page <= total_pages:
    print(f"Scraping product listing page {current_page}...")

    request_params = base_params.copy()
    request_params['page'] = current_page

    try:
        # Use session.get() instead of requests.get() to utilize retry logic
        response = session.get(product_list_base_url, headers=headers, params=request_params)
        response.raise_for_status() 
        data = response.json()

        if current_page == 1:
            print("\n--- FIRST PRODUCT LISTING PAGE RAW JSON RESPONSE (for debugging) ---")
            print(json.dumps(data, indent=2)[:2000])
            print("--------------------------------------------------------------------\n")

        total_count_found = data.get('totalResultCount', 0)
        pagination_info = data.get('pagination', {})
        total_pages_found = pagination_info.get('totalPages', current_page)
        
        products_on_page = data.get('plpList', {}).get('productList', []) 
        
        if current_page == 1:
            total_count = total_count_found
            total_pages = total_pages_found
            if total_count == 0 and total_pages > 0 and len(products_on_page) > 0:
                 total_count = total_pages * base_params['page-size']
            
            print(f"Total products (extracted, potentially estimated): {total_count}, Total pages (extracted): {total_pages}")
            
            if not products_on_page and total_count > 0:
                print("Warning: API returned total products > 0 but no items found on this page via 'plpList.productList'. Exiting.")
                break 

        all_products_raw.extend(products_on_page)

        current_page += 1
        time.sleep(0.5)
        if current_page > total_pages:
            break

    except requests.exceptions.RequestException as e:
        print(f"Error during product listing request on page {current_page}: {e}")
        break 
    except json.JSONDecodeError:
        print(f"Error decoding JSON response on page {current_page}. Response content: {response.text[:500]}...")
        break
    except KeyError as e:
        print(f"Key error in product listing response on page {current_page}: {e}. Response JSON: {json.dumps(data, indent=2)[:500]}...")
        break

print(f"Finished scraping {len(all_products_raw)} raw product entries.")

# Data Flattening 
flattened_products = []
for i, item in enumerate(tqdm(all_products_raw, desc="Fetching Supplier Details", unit="product")):
    product_info = {}

    product_info['id'] = item.get('id') 
    product_info['sku'] = item.get('articleId') 
    if not product_info['sku']:
        product_info['sku'] = item.get('id')
        
    product_info['name'] = item.get('productName') 
    product_info['brandName'] = item.get('brandName') 
    product_info['url'] = item.get('url') 
    product_info['canonical_url'] = f"https://www2.hm.com{item.get('url')}" if item.get('url') else None

    prices_list = item.get('prices', [])
    if prices_list:
        main_price = prices_list[0]
        product_info['final_price_value'] = main_price.get('price')
        formatted_price = main_price.get('formattedPrice', '')
        if formatted_price:
            currency_match = re.match(r'^\W+', formatted_price)
            product_info['final_price_currency'] = currency_match.group(0).strip() if currency_match else None
        else:
            product_info['final_price_currency'] = None

        product_info['regular_price_value'] = main_price.get('price')
        product_info['regular_price_currency'] = product_info['final_price_currency']

    availability_info = item.get('availability', {})
    product_info['in_stock'] = availability_info.get('stockState') == 'Available'
    product_info['coming_soon'] = availability_info.get('comingSoon')

    images_list = item.get('images', [])
    image_urls_parsed = [{'url': img.get('url')} for img in images_list]
    product_info['all_image_data'] = json.dumps(image_urls_parsed)

    swatches_list = item.get('swatches', [])
    swatch_data = []
    for swatch in swatches_list:
        swatch_data.append({
            'articleId': swatch.get('articleId'),
            'url': f"https://www2.hm.com{swatch.get('url')}" if swatch.get('url') else None,
            'colorName': swatch.get('colorName'),
            'colorCode': swatch.get('colorCode'),
            'productImage': swatch.get('productImage')
        })
    product_info['swatch_data'] = json.dumps(swatch_data)
    
    product_info['colorName_main'] = item.get('colorName')
    product_info['colorCode_main'] = item.get('colorCode')
    product_info['colourShades'] = item.get('colourShades')

    sizes_list = item.get('sizes', [])
    size_data = [{'id': size.get('id'), 'label': size.get('label')} for size in sizes_list]
    product_info['available_sizes'] = json.dumps(size_data)

    product_info['mainCatCode'] = item.get('mainCatCode')
    product_info['sellingAttribute'] = item.get('sellingAttribute')
    product_info['newArrival'] = item.get('newArrival')
    product_info['isSale'] = item.get('isSale', False)
    product_info['modelImage'] = item.get('modelImage')
    product_info['colors'] = item.get('colors')

    # Supplier Details
    supplier_country = None
    supplier_name = None
    factory_name = None
    factory_address = None
    workers_number = None

    if product_info['sku']: 
        supplier_detail_url = f"{supplier_detail_base_url}{product_info['sku']}"
        try:
            supplier_response = session.get(supplier_detail_url, headers=headers)
            supplier_response.raise_for_status() 
            supplier_data = supplier_response.json()

            countries = supplier_data.get('countries', [])
            if countries:
                first_country = countries[0]
                supplier_country = first_country.get('name')
                
                suppliers = first_country.get('suppliers', [])
                if suppliers:
                    first_supplier = suppliers[0]
                    supplier_name = first_supplier.get('name')
                    
                    factories = first_supplier.get('factories', [])
                    if factories:
                        first_factory = factories[0]
                        factory_name = first_factory.get('name')
                        factory_address = first_factory.get('address')
                        workers_number = first_factory.get('workersNumber')
            
        except requests.exceptions.RequestException as e:
            tqdm.write(f"Error fetching supplier for SKU {product_info['sku']} after {RETRY_MAX} retries: {e}")
        except json.JSONDecodeError:
            tqdm.write(f"Error decoding supplier JSON for SKU {product_info['sku']}. Response: {supplier_response.text[:100]}...")
        except Exception as e:
            tqdm.write(f"Unexpected error parsing supplier data for SKU {product_info['sku']}: {e}")


    product_info['supplier_country'] = supplier_country
    product_info['supplier_name'] = supplier_name
    product_info['factory_name'] = factory_name
    product_info['factory_address'] = factory_address
    product_info['factory_workers_number'] = workers_number

    flattened_products.append(product_info)

df = pd.DataFrame(flattened_products)
print("\nSample data:")
print(df.head())
print("\nDataFrame columns:")
print(df.columns)

df.to_csv('HnM_CLOTHING_MEN_USA_010725.csv', index=False, encoding='utf-8')
print(f"Data saved to HnM_CLOTHING_MEN_USA_010725.csv. Total products with data: {len(df)}")

Scraping product listing page 1...

--- FIRST PRODUCT LISTING PAGE RAW JSON RESPONSE (for debugging) ---
{
  "requestDateTime": "2025-07-01T13:37:39.281Z",
  "responseSource": "Elevate",
  "pagination": {
    "currentPage": 1,
    "nextPageNum": 2,
    "totalPages": 53
  },
  "plpList": {
    "productList": [
      {
        "id": "1292861001",
        "trackingId": "OzU7IzsxMjkyODYxMDAxOyM7IzsvZmFzaGlvbi9ERVNLVE9QL0xBTkRJTkdfUEFHRS9OQVZJR0FUSU9OX1JFU1VMVDsjOyM7T0JKRUNUSVZFJDsxOzc5OyM7IzsvbWVuL3Nob3AtYnktcHJvZHVjdC92aWV3LWFsbDs_e4",
        "productName": "Slim-Fit Cable-Knit Polo Shirt",
        "external": false,
        "brandName": "H&M",
        "url": "/en_us/productpage.1292861001.html",
        "showPriceMarker": false,
        "prices": [
          {
            "priceType": "whitePrice",
            "price": 44.99,
            "minPrice": 44.99,
            "maxPrice": 44.99,
            "formattedPrice": "$ 44.99"
          }
        ],
        "availability": {
          "s

2025-07-01 20:38:34,739 - WARNING - Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))': /search-services/v1/en_us/listing/resultpage?pageSource=PLP&page=47&sort=RELEVANCE&pageId=%2Fmen%2Fshop-by-product%2Fview-all&page-size=36&categoryId=men_viewall&filters=sale%3Afalse%7C%7ColdSale%3Afalse&touchPoint=DESKTOP&skipStockCheck=false
2025-07-01 20:38:35,787 - WARNING - Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000210EDD04A50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /search-services/v1/en_us/listing/resultpage?pageSource=PLP&page=47&sort=RELEVANCE&pageId=%2Fmen%2Fshop-by-product%2Fview-all&page-size=36&ca

Scraping product listing page 48...
Scraping product listing page 49...
Scraping product listing page 50...
Scraping product listing page 51...
Scraping product listing page 52...
Scraping product listing page 53...
Finished scraping 1892 raw product entries.


Fetching Supplier Details:   0%|                                               | 4/1892 [00:20<2:22:08,  4.52s/product]

Error fetching supplier for SKU 1290111001 after 5 retries: HTTPSConnectionPool(host='www2.hm.com', port=443): Max retries exceeded with url: /en_us/supplierDetails/articles/1290111001 (Caused by ResponseError('too many 400 error responses'))


...

Error fetching supplier for SKU 1235422002 after 5 retries: HTTPSConnectionPool(host='www2.hm.com', port=443): Max retries exceeded with url: /en_us/supplierDetails/articles/1235422002 (Caused by ResponseError('too many 400 error responses'))


Fetching Supplier Details:  99%|███████████████████████████████████████████▋| 1877/1892 [1:06:40<01:51,  7.41s/product]

Error fetching supplier for SKU 1232212003 after 5 retries: HTTPSConnectionPool(host='www2.hm.com', port=443): Max retries exceeded with url: /en_us/supplierDetails/articles/1232212003 (Caused by ResponseError('too many 400 error responses'))


Fetching Supplier Details: 100%|████████████████████████████████████████████| 1892/1892 [1:06:49<00:00,  2.12s/product]


Sample data:
           id         sku                            name brandName  \
0  1292861001  1292861001  Slim-Fit Cable-Knit Polo Shirt       H&M   
1  1290111001  1290111001   Regular-Fit Boxy Poplin Shirt       H&M   
2  1292861003  1292861003  Slim-Fit Cable-Knit Polo Shirt       H&M   
3  1275594004  1275594004             Slim-Fit Crop Pants       H&M   
4  1292861002  1292861002  Slim-Fit Cable-Knit Polo Shirt       H&M   

                                  url  \
0  /en_us/productpage.1292861001.html   
1  /en_us/productpage.1290111001.html   
2  /en_us/productpage.1292861003.html   
3  /en_us/productpage.1275594004.html   
4  /en_us/productpage.1292861002.html   

                                       canonical_url  final_price_value  \
0  https://www2.hm.com/en_us/productpage.12928610...              44.99   
1  https://www2.hm.com/en_us/productpage.12901110...              24.99   
2  https://www2.hm.com/en_us/productpage.12928610...              44.99   
3  https://w